# Blockhash Oracle Configuration Script

This notebook configures the deployed blockhash oracle contracts.
Configuration state is parsed from RPC each time - no state files are maintained.

## 1. Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

# Network type selection
NETWORK_TYPE = "testnets"  # "testnets" or "mainnets"

# Configuration mode
# - "full": Configure all deployed chains and peers
# - "auto": Configure all deployed chains
# - "manual": Only configure chains listed in CHAINS_TO_CONFIGURE
CONFIGURATION_MODE = "full"

# For manual mode, specify which chains to configure
# Example: ["base-sepolia", "optimism-sepolia"]
CHAINS_TO_CONFIGURE = []

# Force reconfiguration options
FORCE_RECONFIGURE_PEERS = False  # Force re-set all peer relationships
FORCE_RECONFIGURE_DVNS = False  # Force re-set all DVN configurations

SET_SEND_LIBS = False
# DVN Configuration
# Set to True to configure DVNs (requires proper delegate permissions)
CONFIGURE_DVNS = True

# Curve Finance DVN addresses (optional custom DVN)
CURVE_DVNS = {
    "sepolia": "0x3a8bf25ff10ec52dc7efe32aafaef84072fdcf8c",
    "base-sepolia": "0xfe3c4c5676c04a4ebd9961a7c5934be16beb35df",
    "optimism-sepolia": "0x75d7ad554475008cae51298578cda6936c432d4e",
    "arbitrum-sepolia": "0x4b916807a527fdaa66b3bff5a5307f5129b60f43",
}

## 2. Initialize Environment

In [ ]:
import json
import os
import sys
import logging
import subprocess
import time
from pathlib import Path

import boa
from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3
from eth_abi import decode
import eth_abi

# Add parent directory to path for imports
sys.path.append(str(Path().resolve().parent))
from ABIs import endpointV2_abi

# Import from deployment folder
from LZMetadata import LZMetadata
from DeploymentManager import DeploymentManager

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()

# Constants
READ_CHANNEL_ID = 4294967295  # max uint32
CONFIG_TYPE_READ = 1
CONFIG_TYPE_ULN = 2
CONFIG_TYPE_EXECUTOR = 1

## 3. Helper Functions

In [ ]:
def get_vyper_abi(filepath):
    """Get ABI from Vyper contract file"""
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


def send_tx_single(w3, func, acc, value=0, gas=0):
    """Send a single transaction"""
    tx = func.build_transaction(
        {
            "from": acc.address,
            "nonce": w3.eth.get_transaction_count(acc.address),
            "value": value,
            "gas": gas,
        }
    )
    if gas > 0:
        tx["gas"] = gas
    else:
        tx["gas"] = int(w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=acc.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0, gas=0):
    """Send transaction with retry logic"""
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value, gas)
            success = True
        except Exception as e:
            if (
                "replacement transaction underpriced" in str(e)
                or "nonce too low" in str(e)
                or "could not replace existing tx" in str(e)
            ):
                print(str(e), "Retrying...")
                success = False
                time.sleep(1)
            else:
                raise e
    return tx_hash


def inject_curve_dvns(dvns_list, chain_key):
    """Add Curve Finance DVN to DVN list if available"""
    if chain_key not in CURVE_DVNS:
        return dvns_list

    curve_dvn = {
        "address": CURVE_DVNS[chain_key],
        "version": 2,
        "canonicalName": "Curve Finance",
        "id": "curve",
        "lzReadCompatible": True,
    }

    if not any(d["address"].lower() == curve_dvn["address"].lower() for d in dvns_list):
        dvns_list.append(curve_dvn)

    return dvns_list


def checksum(address):
    """Convert address to checksum format"""
    return Web3.to_checksum_address(address)


def decode_dvn_config(hex_data, config_type="uln"):
    """Decode DVN configuration from hex data"""
    if not hex_data or hex_data == "0x":
        return None

    # if hex_data is string - make it bytes
    if isinstance(hex_data, str):
        hex_data = bytes.fromhex(hex_data.replace("0x", ""))

    try:
        if config_type == "read":
            # Read config structure
            decoded = decode(["(address,uint8,uint8,uint8,address[],address[])"], hex_data)
            return {
                "executor": decoded[0][0],
                "requiredDVNCount": decoded[0][1],
                "optionalDVNCount": decoded[0][2],
                "optionalDVNThreshold": decoded[0][3],
                "requiredDVNs": [checksum(addr) for addr in decoded[0][4]],
                "optionalDVNs": [checksum(addr) for addr in decoded[0][5]],
            }
        elif config_type == "executor":
            # Executor config
            decoded = decode(["(uint32,address)"], hex_data)
            return {"executor": decoded[0][1]}
        else:
            # ULN config structure
            decoded = decode(["(uint64,uint8,uint8,uint8,address[],address[])"], hex_data)

            return {
                "confirmations": decoded[0][0],
                "requiredDVNCount": decoded[0][1],
                "optionalDVNCount": decoded[0][2],
                "optionalDVNThreshold": decoded[0][3],
                "requiredDVNs": [checksum(addr) for addr in decoded[0][4]],
                "optionalDVNs": [checksum(addr) for addr in decoded[0][5]],
            }
    except Exception as e:
        logging.warning(f"Failed to decode DVN config: {e}")
        return None

## 4. Load Deployment State

In [ ]:
# Load deployment state
deployment_manager = DeploymentManager()
deployed_contracts = deployment_manager.get_all_deployed_contracts(NETWORK_TYPE)

if not deployed_contracts:
    raise ValueError(f"No deployments found for {NETWORK_TYPE}")

logging.info(f"Found deployments for {len(deployed_contracts)} chains")

# Load chains configuration
with open("chains.json", "r") as f:
    chains_config = json.load(f)

all_chains = chains_config[NETWORK_TYPE]

# Find main chain
main_chain = None
for chain_name, config in all_chains.items():
    if config.get("is_main_chain", False):
        main_chain = chain_name
        break

if not main_chain:
    raise ValueError(f"No main chain defined for {NETWORK_TYPE}")

logging.info(f"Main chain: {main_chain}")

# Determine which chains to configure
if CONFIGURATION_MODE == "full":
    # Configure all deployed chains
    chains_to_configure = set(deployed_contracts.keys())
elif CONFIGURATION_MODE == "manual":
    # Configure only specified chains
    chains_to_configure = set(CHAINS_TO_CONFIGURE)
    # Verify all specified chains have deployments
    for chain in chains_to_configure:
        if chain not in deployed_contracts:
            raise ValueError(f"Chain {chain} specified but no deployment found")
else:
    raise ValueError(f"Invalid CONFIGURATION_MODE: {CONFIGURATION_MODE}")

# Filter chains
chains = {k: v for k, v in all_chains.items() if k in chains_to_configure}

logging.info(f"Chains to configure: {list(chains.keys())}")

## 5. Setup Account

In [ ]:
# Get deployer account
if NETWORK_TYPE == "testnets":
    private_key = os.environ.get("WEB3_TESTNET_PK")
    if not private_key:
        raise ValueError("WEB3_TESTNET_PK not found in environment")
    account = Account.from_key(private_key)
else:
    # For mainnets, use secure key utilities
    sys.path.append(os.path.expanduser("~/projects/keys/scripts"))
    from secure_key_utils import get_account_pk

    account = get_account_pk()

logging.info(f"Account address: {account.address}")

## 6. Initialize State with Deployed Contracts

In [ ]:
# Initialize LayerZero metadata
lz = LZMetadata()

# Initialize state dictionary
state_dict = {}

# Get ABIs
ABI_RELAY = get_vyper_abi("../../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../../contracts/BlockOracle.vy")
ABI_HEADER_VERIFIER = get_vyper_abi("../../contracts/HeaderVerifier.vy")

# Load all deployed chains (including main chain if needed for read config)
all_deployed_chains = set(deployed_contracts.keys())
if main_chain in all_deployed_chains and main_chain not in chains:
    chains[main_chain] = all_chains[main_chain]

for chain_name, config in chains.items():
    if chain_name not in deployed_contracts:
        logging.warning(f"No contracts deployed on {chain_name}, skipping")
        continue

    state_dict[chain_name] = {}

    # Build RPC URL
    if "rpc" in config:
        rpc_url = config["rpc"]
    else:
        api_key = os.environ.get(config["rpc_env_var"])
        if not api_key:
            raise ValueError(f"{config['rpc_env_var']} not found in environment")
        rpc_url = config["rpc_template"].format(api_key)

    # Setup boa environment
    boa.set_network_env(rpc_url)
    boa.env.add_account(account)

    # Store state
    state_dict[chain_name]["config"] = config
    state_dict[chain_name]["rpc"] = rpc_url
    state_dict[chain_name]["boa"] = boa.env
    state_dict[chain_name]["w3"] = Web3(Web3.HTTPProvider(rpc_url))
    state_dict[chain_name]["evm_version"] = config["evm_version"]

    # Get LayerZero metadata
    try:
        lz_metadata = lz.get_chain_metadata(chain_name)
        state_dict[chain_name]["eid"] = lz_metadata["metadata"]["eid"]
        state_dict[chain_name]["endpoint"] = lz_metadata["metadata"]["endpointV2"]
        state_dict[chain_name]["send_lib"] = lz_metadata["metadata"].get(
            "sendUln302", "unavailable"
        )
        state_dict[chain_name]["receive_lib"] = lz_metadata["metadata"].get(
            "receiveUln302", "unavailable"
        )
        state_dict[chain_name]["read_lib"] = lz_metadata["metadata"].get(
            "readLib1002", "unavailable"
        )
        state_dict[chain_name]["dvns"] = lz_metadata["dvns"]
        state_dict[chain_name]["executor"] = lz_metadata["metadata"].get(
            "executor", "0x0000000000000000000000000000000000000000"
        )
    except Exception as e:
        logging.warning(f"Failed to get LZ metadata for {chain_name}: {e}")

    # Load deployed contracts
    contracts = deployed_contracts[chain_name]

    # Load contracts using boa
    with boa.swap_env(state_dict[chain_name]["boa"]):
        logging.info(f"Loading contracts for {chain_name}")
        if chain_name == main_chain:
            # Main chain has MainnetBlockView
            view_deployer = boa.load_partial(
                "../../contracts/MainnetBlockView.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["oracle"] = view_deployer.at(contracts["MainnetBlockView"])
        else:
            # Other chains have BlockOracle, HeaderVerifier, and LZBlockRelay
            oracle_deployer = boa.load_partial(
                "../../contracts/BlockOracle.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["oracle"] = oracle_deployer.at(contracts["BlockOracle"])

            verifier_deployer = boa.load_partial(
                "../../contracts/HeaderVerifier.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["header_verifier"] = verifier_deployer.at(
                contracts["HeaderVerifier"]
            )

            relay_deployer = boa.load_partial(
                "../../contracts/messengers/LZBlockRelay.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["block_relay"] = relay_deployer.at(contracts["LZBlockRelay"])

            # Also setup Web3 contracts for delegate operations
            state_dict[chain_name]["block_relay_w3"] = state_dict[chain_name]["w3"].eth.contract(
                address=contracts["LZBlockRelay"], abi=ABI_RELAY
            )
            state_dict[chain_name]["oracle_w3"] = state_dict[chain_name]["w3"].eth.contract(
                address=contracts["BlockOracle"], abi=ABI_ORACLE
            )
            state_dict[chain_name]["header_verifier_w3"] = state_dict[chain_name][
                "w3"
            ].eth.contract(address=contracts["HeaderVerifier"], abi=ABI_HEADER_VERIFIER)
            state_dict[chain_name]["endpoint_w3"] = state_dict[chain_name]["w3"].eth.contract(
                address=state_dict[chain_name]["endpoint"], abi=endpointV2_abi
            )

logging.info("Loaded all deployed contracts")

## 7. Configure Peers

In [ ]:
# Configure peers for all block relay contracts
# This checks existing configuration from RPC and only sets what's needed

for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue

    # Use dict to manage unique peer relationships
    desired_peers = {}

    # 1. If read-enabled: add all other chains as peers for broadcasting
    if state_dict[chain_name]["read_lib"] != "unavailable":
        for target_key in all_deployed_chains:
            if target_key != main_chain and target_key != chain_name:
                if target_key in state_dict:
                    desired_peers[state_dict[target_key]["eid"]] = state_dict[target_key][
                        "block_relay"
                    ].address

    # 2. For all chains: add read-enabled chains as peers (to receive their broadcasts)
    for source_key in all_deployed_chains:
        if (
            source_key != main_chain
            and source_key != chain_name
            and source_key in state_dict
            and state_dict[source_key]["read_lib"] != "unavailable"
        ):
            desired_peers[state_dict[source_key]["eid"]] = state_dict[source_key][
                "block_relay"
            ].address

    if not desired_peers:
        logging.info(f"No peers needed for {chain_name}")
        continue

    with boa.swap_env(state_dict[chain_name]["boa"]):
        relay_contract = state_dict[chain_name]["block_relay"]

        # Check which peers need to be set
        to_set_eids = []
        to_set_peers = []

        for peer_eid, peer_address in desired_peers.items():
            current_peer = relay_contract.peers(peer_eid)

            # Check if peer needs to be set or updated
            if (
                FORCE_RECONFIGURE_PEERS
                or peer_address[2:].lower() not in str(current_peer.hex()).lower()
            ):
                to_set_eids.append(peer_eid)
                to_set_peers.append(peer_address)

        # Set peers if needed
        if len(to_set_eids) > 0:
            logging.info(f"Setting {len(to_set_eids)} peers on {chain_name}...")
            relay_contract.set_peers(to_set_eids, to_set_peers)
            logging.info(f"Set {len(to_set_eids)} peers on {chain_name}")
        else:
            logging.info(f"All {len(desired_peers)} peers already configured on {chain_name}")

## 8. Configure Relay-Oracle-HeaderVerifier

In [ ]:
# Configure block oracles and committers
for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue

    with boa.swap_env(state_dict[chain_name]["boa"]):
        relay_contract = state_dict[chain_name]["block_relay"]
        oracle_contract = state_dict[chain_name]["oracle"]
        header_verifier = state_dict[chain_name]["header_verifier"]

        # Set block oracle on relay
        if relay_contract.block_oracle() != oracle_contract.address:
            relay_contract.set_block_oracle(oracle_contract.address)
            logging.info(f"Set block oracle on {chain_name}")
        else:
            logging.info(f"Block oracle already set on {chain_name}")

        # Add relay as committer to oracle
        if not oracle_contract.is_committer(relay_contract.address):
            oracle_contract.add_committer(relay_contract.address, True)
            logging.info(f"Added relay as committer on {chain_name}")
        else:
            logging.info(f"Relay already a committer on {chain_name}")

        # Set header verifier on oracle
        if oracle_contract.header_verifier() != header_verifier.address:
            oracle_contract.set_header_verifier(header_verifier.address)
            logging.info(f"Set header verifier on {chain_name}")
        else:
            logging.info(f"Header verifier already set on {chain_name}")

## 9. Configure Read Settings

In [ ]:
# Configure read settings for read-enabled chains
for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue

    if state_dict[chain_name]["read_lib"] != "unavailable":
        with boa.swap_env(state_dict[chain_name]["boa"]):
            relay_contract = state_dict[chain_name]["block_relay"]

            # Check current read config
            is_enabled = relay_contract.read_enabled()
            read_channel = relay_contract.read_channel()
            mainnet_eid = relay_contract.mainnet_eid()
            mainnet_view = relay_contract.mainnet_block_view()

            expected_mainnet_view = state_dict[main_chain]["oracle"].address
            expected_mainnet_eid = state_dict[main_chain]["eid"]

            if (
                not is_enabled
                or read_channel != READ_CHANNEL_ID
                or mainnet_eid != expected_mainnet_eid
                or mainnet_view != expected_mainnet_view
            ):
                logging.info(f"Setting read config on {chain_name}...")
                relay_contract.set_read_config(
                    True,
                    READ_CHANNEL_ID,
                    expected_mainnet_eid,
                    expected_mainnet_view,
                )
                logging.info(f"Read config set on {chain_name}")
            else:
                logging.info(f"Read config already set correctly on {chain_name}")

## 10. Configure LayerZero Libraries

In [ ]:
# Configure LayerZero send/receive libraries via endpoint delegate
all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]

for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue

    # Get configured peers
    relay_w3 = state_dict[chain_name]["block_relay_w3"]
    endpoint_w3 = state_dict[chain_name]["endpoint_w3"]
    w3 = state_dict[chain_name]["w3"]

    eid_peers = {}
    for eid in all_eids:
        peer = relay_w3.functions.peers(eid).call().hex()
        if peer != "0000000000000000000000000000000000000000000000000000000000000000":
            eid_peers[eid] = "0x" + peer[24:]

    logging.info(f"\nConfiguring libraries on {chain_name}...")
    logging.info(f"Peers: {len(eid_peers)}")

    send_lib = state_dict[chain_name]["send_lib"]
    receive_lib = state_dict[chain_name]["receive_lib"]
    read_lib = state_dict[chain_name]["read_lib"]
    oapp = relay_w3.address

    # Configure send/receive libs for each peer
    for eid, _ in eid_peers.items():
        # Check and set send library
        if SET_SEND_LIBS:
            try:
                current_send_lib = endpoint_w3.functions.getSendLibrary(oapp, eid).call()
                if current_send_lib != send_lib:
                    call_fn = endpoint_w3.functions.setSendLibrary(oapp, eid, send_lib)
                    tx_hash = send_tx(w3, call_fn, account)
                    logging.info(f"Set send library for EID {eid}: {tx_hash.hex()}")
            except Exception as e:
                logging.error(f"Error setting send library on {chain_name} for EID {eid}: {e}")

            # Check and set receive library
            try:
                current_receive_lib = endpoint_w3.functions.getReceiveLibrary(oapp, eid).call()[0]
                if current_receive_lib != receive_lib:
                    call_fn = endpoint_w3.functions.setReceiveLibrary(oapp, eid, receive_lib, 0)
                    tx_hash = send_tx(w3, call_fn, account)
                    logging.info(f"Set receive library for EID {eid}: {tx_hash.hex()}")
            except Exception as e:
                logging.error(f"Error setting receive library on {chain_name} for EID {eid}: {e}")

    # Configure read library if available
    if read_lib != "unavailable":
        try:
            read_channel = relay_w3.functions.read_channel().call()
            if read_channel != 0:
                # Check send library for read channel
                current_send_lib = endpoint_w3.functions.getSendLibrary(oapp, read_channel).call()
                if current_send_lib != read_lib:
                    call_fn = endpoint_w3.functions.setSendLibrary(oapp, read_channel, read_lib)
                    tx_hash = send_tx(w3, call_fn, account)
                    logging.info(f"Set send-read library: {tx_hash.hex()}")
                else:
                    logging.info(f"Send-read library already set on {chain_name}")

                # Check receive library for read channel
                current_receive_lib = endpoint_w3.functions.getReceiveLibrary(
                    oapp, read_channel
                ).call()[0]
                if current_receive_lib != read_lib:
                    call_fn = endpoint_w3.functions.setReceiveLibrary(
                        oapp, read_channel, read_lib, 0
                    )
                    tx_hash = send_tx(w3, call_fn, account)
                    logging.info(f"Set receive-read library: {tx_hash.hex()}")
                else:
                    logging.info(f"Receive-read library already set on {chain_name}")
            else:
                logging.info(f"Read channel not set on {chain_name}")
        except Exception as e:
            logging.error(f"Error setting read library on {chain_name}: {e}")

## 11. Analyze DVN Configuration

In [ ]:
# Analyze current DVN configuration from RPC
if CONFIGURE_DVNS:
    logging.info("\n" + "=" * 80)
    logging.info("Analyzing current DVN configurations...")
    logging.info("=" * 80)

    # Build eid to name mapping
    eid_to_name = {state_dict[key]["eid"]: key for key in state_dict.keys()}

    # Analyze each chain
    for chain_name in chains_to_configure:
        if chain_name == main_chain:
            continue

        logging.info(f"\nAnalyzing {chain_name}...")
        relay_w3 = state_dict[chain_name]["block_relay_w3"]
        endpoint_w3 = state_dict[chain_name]["endpoint_w3"]
        oapp = checksum(relay_w3.address)

        # Get configured EIDs
        all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]
        all_eids.append(4294967295)
        eid_peers = {}
        for eid in all_eids:
            peer = relay_w3.functions.peers(eid).call().hex()
            if peer != "0000000000000000000000000000000000000000000000000000000000000000":
                eid_peers[eid] = "0x" + peer[24:]
        configured_eids = eid_peers.keys()
        print(f"Configured EIDs: {configured_eids}")
        # Initialize desired config
        state_dict[chain_name]["desired_dvn_config"] = {}
        state_dict[chain_name]["current_dvn_config"] = {}

        # Get DVNs for this chain (with optional Curve DVN)
        dvns = inject_curve_dvns(state_dict[chain_name]["dvns"].copy(), chain_name)
        print(f"DVNs: {dvns}")

        for eid in configured_eids:
            is_read_channel = eid > 4294965694
            peer_name = (
                "read_channel" if is_read_channel else eid_to_name.get(eid, f"unknown-{eid}")
            )

            # Get current configuration from RPC
            current_config = {"send": None, "receive": None}

            if is_read_channel:
                # For read channel, only check send config
                if state_dict[chain_name]["read_lib"] != "unavailable":
                    try:
                        config_bytes = endpoint_w3.functions.getConfig(
                            oapp, state_dict[chain_name]["read_lib"], eid, CONFIG_TYPE_READ
                        ).call()
                        current_config["send"] = decode_dvn_config(config_bytes, "read")
                    except Exception as e:
                        logging.debug(f"Failed to get read config: {e}")
            else:
                # For regular peers, check both send and receive
                # Send config
                try:
                    config_bytes = endpoint_w3.functions.getConfig(
                        oapp, state_dict[chain_name]["send_lib"], eid, CONFIG_TYPE_ULN
                    ).call()
                    current_config["send"] = decode_dvn_config(config_bytes, "uln")
                except Exception as e:
                    logging.debug(f"Failed to get send config: {e}")

                # Receive config
                try:
                    config_bytes = endpoint_w3.functions.getConfig(
                        oapp, state_dict[chain_name]["receive_lib"], eid, CONFIG_TYPE_ULN
                    ).call()
                    current_config["receive"] = decode_dvn_config(config_bytes, "uln")
                except Exception as e:
                    logging.debug(f"Failed to get receive config: {e}")
            state_dict[chain_name]["current_dvn_config"][eid] = current_config

            # Build desired configuration
            if is_read_channel and state_dict[chain_name]["read_lib"] != "unavailable":
                # Get read-compatible DVNs
                read_dvns = [
                    dvn
                    for dvn in dvns
                    if dvn.get("lzReadCompatible", False)
                    and dvn["id"] in ["layerzero-labs", "nethermind"]  # , "curve"]
                ]
                if read_dvns:
                    state_dict[chain_name]["desired_dvn_config"][eid] = {
                        "send": {
                            "requiredDVNs": [dvn["address"] for dvn in read_dvns],
                            "optionalDVNs": [],
                            "executor": state_dict[chain_name]["executor"],
                        },
                        "receive": {"requiredDVNs": [], "optionalDVNs": []},
                    }
            else:
                # Regular peer configuration
                peer_chain = eid_to_name.get(eid)
                if peer_chain and peer_chain in state_dict:
                    peer_dvns = inject_curve_dvns(state_dict[peer_chain]["dvns"].copy(), peer_chain)

                    # Get DVNs by ID (only non-read-compatible for ULN)
                    my_dvns_by_id = {
                        d["id"]: d["address"]
                        for d in dvns
                        if not d.get("lzReadCompatible", False) or d["id"] == "curve"
                    }
                    peer_dvns_by_id = {
                        d["id"]: d["address"]
                        for d in peer_dvns
                        if not d.get("lzReadCompatible", False) or d["id"] == "curve"
                    }

                    # Get required DVNs
                    required_ids = []
                    for dvn_id in ["layerzero-labs", "nethermind", "curve"]:
                        if dvn_id in my_dvns_by_id and dvn_id in peer_dvns_by_id:
                            required_ids.append(dvn_id)

                    if required_ids:
                        # Build desired config
                        state_dict[chain_name]["desired_dvn_config"][eid] = {
                            "send": {
                                "requiredDVNs": [my_dvns_by_id[id] for id in required_ids],
                                "optionalDVNs": [],
                                "executor": state_dict[chain_name]["executor"],
                            },
                            "receive": {"requiredDVNs": [], "optionalDVNs": []},
                        }

                        # Also set receive config on peer chain
                        if "desired_dvn_config" not in state_dict[peer_chain]:
                            state_dict[peer_chain]["desired_dvn_config"] = {}

                        if state_dict[chain_name]["eid"] not in state_dict[peer_chain].get(
                            "desired_dvn_config", {}
                        ):
                            state_dict[peer_chain]["desired_dvn_config"][
                                state_dict[chain_name]["eid"]
                            ] = {
                                "send": {"requiredDVNs": [], "optionalDVNs": []},
                                "receive": {"requiredDVNs": [], "optionalDVNs": []},
                            }

                        state_dict[peer_chain]["desired_dvn_config"][state_dict[chain_name]["eid"]][
                            "receive"
                        ] = {
                            "requiredDVNs": [peer_dvns_by_id[id] for id in required_ids],
                            "optionalDVNs": [],
                        }

    # Print analysis summary
    for chain_name in chains_to_configure:
        if chain_name == main_chain:
            continue

        print(f"\n{chain_name} DVN configuration analysis:")
        for eid, config in state_dict[chain_name].get("current_dvn_config", {}).items():
            peer = eid_to_name.get(eid, eid)
            if eid > 4294965694:
                peer = "read_channel"
            print(f"\nPeer {peer} (EID {eid}):")

            # Show current config
            if config["send"]:
                print(
                    f"  Current Send DVNs: {len(config['send'].get('requiredDVNs', []))} required"
                )
            else:
                print("  Current Send DVNs: Not configured")

            if config["receive"]:
                print(
                    f"  Current Receive DVNs: {len(config['receive'].get('requiredDVNs', []))} required"
                )
            elif eid <= 4294965694:  # Only show receive for non-read channels
                print("  Current Receive DVNs: Not configured")

            # Show desired config
            desired = state_dict[chain_name].get("desired_dvn_config", {}).get(eid)
            if desired:
                if desired["send"]["requiredDVNs"]:
                    print(f"  Desired Send DVNs: {len(desired['send']['requiredDVNs'])} required")
                if desired["receive"]["requiredDVNs"]:
                    print(
                        f"  Desired Receive DVNs: {len(desired['receive']['requiredDVNs'])} required"
                    )

else:
    logging.info("Skipping DVN configuration analysis (CONFIGURE_DVNS=False)")

## 12. Apply DVN Configuration

In [ ]:
FORCE_RECONFIGURE_DVNS = False
if CONFIGURE_DVNS:
    logging.info("\n" + "=" * 80)
    logging.info("Applying DVN configurations...")
    logging.info("=" * 80)

    # Apply all configurations
    for chain_name in chains_to_configure:
        if chain_name == main_chain:
            continue

        logging.info(f"\nSetting DVN configs for {chain_name}...")
        relay_w3 = state_dict[chain_name]["block_relay_w3"]
        endpoint_w3 = state_dict[chain_name]["endpoint_w3"]
        w3 = state_dict[chain_name]["w3"]
        oapp = checksum(relay_w3.address)

        # Get libraries
        libs = {
            "send": checksum(state_dict[chain_name]["send_lib"])
            if state_dict[chain_name]["send_lib"] != "unavailable"
            else None,
            "receive": checksum(state_dict[chain_name]["receive_lib"])
            if state_dict[chain_name]["receive_lib"] != "unavailable"
            else None,
            "read": checksum(state_dict[chain_name]["read_lib"])
            if state_dict[chain_name]["read_lib"] != "unavailable"
            else None,
        }

        # For each peer relationship
        for eid, desired_config in state_dict[chain_name].get("desired_dvn_config", {}).items():
            is_read_channel = eid > 4294965694
            peer_name = "read_channel" if is_read_channel else eid_to_name.get(eid)
            current_config = state_dict[chain_name]["current_dvn_config"].get(eid, {})

            # Process send and receive configs
            for config_type in ["send", "receive"]:
                if not desired_config[config_type]["requiredDVNs"]:
                    continue

                # Skip receive for read channel
                if is_read_channel and config_type == "receive":
                    continue

                # Check if configuration is needed
                current = current_config.get(config_type, {})
                desired = desired_config.get(config_type, {})
                required_same = set(a.lower() for a in current.get("requiredDVNs", [])) == set(
                    a.lower() for a in desired["requiredDVNs"]
                )
                optional_same = set(a.lower() for a in current.get("optionalDVNs", [])) == set(
                    a.lower() for a in desired.get("optionalDVNs", [])
                )
                # Compare configurations
                needs_update = (
                    FORCE_RECONFIGURE_DVNS or not current or not required_same or not optional_same
                )

                if not needs_update:
                    logging.info(
                        f"DVN {config_type} config already set for {chain_name} <-> {peer_name}"
                    )
                    continue

                logging.info(f"\nConfiguring {chain_name} <-> {peer_name} ({config_type})")

                # Determine correct lib and config type
                lib = libs["read"] if is_read_channel else libs[config_type]
                if not lib:
                    continue

                config_type_enum = CONFIG_TYPE_READ if is_read_channel else CONFIG_TYPE_ULN

                # Prepare DVN lists
                required_dvns = sorted([checksum(addr) for addr in desired["requiredDVNs"]])
                optional_dvns = sorted([checksum(addr) for addr in desired.get("optionalDVNs", [])])
                optional_threshold = len(optional_dvns) if optional_dvns else 0

                # Get executor for send configs
                executor = checksum(
                    desired.get("executor", "0x0000000000000000000000000000000000000000")
                )
                # use default executor
                executor = "0x0000000000000000000000000000000000000000"
                logging.info(f"Setting {config_type} config on {chain_name} for {peer_name}:")
                logging.info(f"- Library: {lib}")
                logging.info(f"- Required DVNs: {required_dvns}")
                logging.info(f"- Optional DVNs: {optional_dvns}")
                logging.info(f"- Optional threshold: {optional_threshold}")

                # Build config based on type
                if is_read_channel:
                    # Read config structure
                    config_struct = (
                        executor,  # executor address
                        len(required_dvns),  # required_dvn_count
                        len(optional_dvns),  # optional_dvn_count
                        optional_threshold,  # optional_dvn_threshold
                        required_dvns,  # required_dvns
                        optional_dvns,  # optional_dvns
                    )
                    config_bytes = eth_abi.encode(
                        ["(address,uint8,uint8,uint8,address[],address[])"], [config_struct]
                    )
                else:
                    # ULN config structure for send/receive
                    config_struct = (
                        0,  # confirmations (uint64)
                        len(required_dvns),  # required_dvn_count
                        len(optional_dvns),  # optional_dvn_count
                        optional_threshold,  # optional_dvn_threshold
                        required_dvns,  # required_dvns
                        optional_dvns,  # optional_dvns
                    )
                    config_bytes = eth_abi.encode(
                        ["(uint64,uint8,uint8,uint8,address[],address[])"], [config_struct]
                    )

                # Set executor config separately for ULN (send only)
                if (
                    config_type == "send"
                    and not is_read_channel
                    and executor != "0x0000000000000000000000000000000000000000"
                ):
                    executor_config = (eid, executor)
                    executor_bytes = eth_abi.encode(["(uint32,address)"], [executor_config])
                    executor_param = (eid, CONFIG_TYPE_EXECUTOR, executor_bytes)

                    try:
                        func = endpoint_w3.functions.setConfig(
                            _oapp=oapp, _lib=lib, _params=[executor_param]
                        )
                        tx_hash = send_tx(w3, func, account)
                        logging.info(f"Set executor config: {tx_hash.hex()}")
                    except Exception as e:
                        logging.error(f"Failed to set executor config: {str(e)}")

                # Prepare config parameter
                config_param = (eid, config_type_enum, config_bytes)

                try:
                    # Set config via endpoint
                    func = endpoint_w3.functions.setConfig(
                        _oapp=oapp, _lib=lib, _params=[config_param]
                    )
                    tx_hash = send_tx(w3, func, account)
                    logging.info(
                        f"Set {'read' if is_read_channel else config_type} DVN config: {tx_hash.hex()}"
                    )

                    time.sleep(2)
                except Exception as e:
                    logging.error(f"Failed to set config on {chain_name} for {peer_name}: {str(e)}")
else:
    logging.info("\nDVN configuration skipped")

## 13. Configuration Summary

In [ ]:
# Print configuration summary
print("\n" + "=" * 80)
print("CONFIGURATION SUMMARY")
print("=" * 80)
print(f"Network Type: {NETWORK_TYPE}")
print(f"Configuration Mode: {CONFIGURATION_MODE}")
print(f"Configured chains: {len(chains_to_configure)}")

# Show all deployed chains
print(f"\nAll deployed chains ({len(deployed_contracts)}):")
for chain_name in sorted(deployed_contracts.keys()):
    configured_marker = " [CONFIGURED]" if chain_name in chains_to_configure else ""
    print(f"  - {chain_name}{configured_marker}")

print("\nConfiguration details:")
for chain_name in sorted(chains_to_configure):
    if chain_name == main_chain:
        print(f"\n{chain_name} (Main Chain):")
        print(f"  MainnetBlockView: {state_dict[chain_name]['oracle'].address}")
    else:
        print(f"\n{chain_name}:")
        print(f"  BlockOracle: {state_dict[chain_name]['oracle'].address}")
        print(f"  HeaderVerifier: {state_dict[chain_name]['header_verifier'].address}")
        print(f"  LZBlockRelay: {state_dict[chain_name]['block_relay'].address}")

        # Check peer count
        with boa.swap_env(state_dict[chain_name]["boa"]):
            configured_eids = state_dict[chain_name]["block_relay"].get_configured_eids()
            print(f"  Configured peers: {len(configured_eids)}")

            # Check if read enabled
            if state_dict[chain_name]["read_lib"] != "unavailable":
                read_config = state_dict[chain_name]["block_relay"].read_config()
                if read_config[0]:  # is_enabled
                    print(f"  Read enabled: Yes (channel {read_config[1]})")

print("\n" + "=" * 80)
print("\nConfiguration complete! The oracle system is ready for use.")
print("\nNext step: Run test.ipynb to test the oracle functionality")

In [ ]:
def get_configured_eids(w3, contract):
    """
    Parses PeerSet logs to find all configured peer EIDs.
    A peer is considered configured if it's set to a non-zero address.
    """
    event_filter = contract.events.PeerSet.create_filter(from_block="0x0")
    logs = event_filter.get_all_entries()
    print(logs)


get_configured_eids(state_dict["base-sepolia"]["w3"], state_dict["base-sepolia"]["block_relay_w3"])